In [1]:
import numpy as np
import json

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer,SnowballStemmer

from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.utils import simple_preprocess

import gensim
import pandas as pd

In [2]:
with open("annotations_trainval2017/annotations/captions_val2017.json") as file:
    data_val=json.load(file)

In [3]:
print(len(data_val))
#print(data_val["info"])
#print(data_val["annotations"])
for i in data_val.keys():
    print(i,len(i))

4
info 4
licenses 8
images 6
annotations 11


In [4]:
print(data_val["images"])

[{'license': 4, 'file_name': '000000397133.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-14 17:02:52', 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg', 'id': 397133}, {'license': 1, 'file_name': '000000037777.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000037777.jpg', 'height': 230, 'width': 352, 'date_captured': '2013-11-14 20:55:31', 'flickr_url': 'http://farm9.staticflickr.com/8429/7839199426_f6d48aa585_z.jpg', 'id': 37777}, {'license': 4, 'file_name': '000000252219.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000252219.jpg', 'height': 428, 'width': 640, 'date_captured': '2013-11-14 22:32:02', 'flickr_url': 'http://farm4.staticflickr.com/3446/3232237447_13d84bd0a1_z.jpg', 'id': 252219}, {'license': 1, 'file_name': '000000087038.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000087038.jpg', 'height': 480, 'width': 640, 'date_c

In [32]:
print(data_val["info"])
#print(data_val["licenses"])

{'description': 'COCO 2017 Dataset', 'url': 'http://cocodataset.org', 'version': '1.0', 'year': 2017, 'contributor': 'COCO Consortium', 'date_created': '2017/09/01'}


In [6]:
print(len(data_val['annotations']))
print(data_val['annotations'][1811])

25014
{'image_id': 397133, 'id': 370509, 'caption': 'A man is in a kitchen making pizzas.'}


In [7]:
count=0
for images in data_val['annotations']:
    count +=1
    if images['image_id'] == 397133:
        print(count-1)
        print(images['caption'])

1811
A man is in a kitchen making pizzas.
1812
Man in apron standing on front of oven with pans and bakeware
1819
A baker is working in the kitchen rolling dough.
1822
A person standing by a stove in a kitchen.
1842
A table with pies being made and a person standing near a wall with pots and pans hanging on the wall.


In [8]:
stemmer = SnowballStemmer('english')

In [9]:
def lemmatize_stemming(text):
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    #print(text)
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
count=0
for images in data_val['annotations']:
    if images['image_id'] == 179765:
        result=preprocess(images['caption'])
        #count+=1
        print(result)
        
#print(count)

['black', 'honda', 'motorcycle', 'park', 'garage']
['honda', 'motorcycle', 'park', 'grass', 'driveway']
['black', 'honda', 'motorcycle', 'dark', 'burgundy', 'seat']
['motorcycle', 'park', 'gravel', 'garage']
['motorcycle', 'brake', 'extend', 'stand', 'outside']


In [11]:
doc_set = []

In [12]:
data_dict={}
for image in data_val['annotations']:
    if image["image_id"] in data_dict:
        data_dict[image["image_id"]].append(image["caption"])
    else:
        data_dict.update({image["image_id"]:[image["caption"]]})


In [13]:
print(len(data_dict)) ### image_id(key) -- > text_corpus(value)

5000


In [14]:
print(data_dict[179765])

['A black Honda motorcycle parked in front of a garage.', 'A Honda motorcycle parked in a grass driveway', 'A black Honda motorcycle with a dark burgundy seat.', 'Ma motorcycle parked on the gravel in front of a garage', 'A motorcycle with its brake extended standing outside']


In [15]:
print(data_dict)

{179765: ['A black Honda motorcycle parked in front of a garage.', 'A Honda motorcycle parked in a grass driveway', 'A black Honda motorcycle with a dark burgundy seat.', 'Ma motorcycle parked on the gravel in front of a garage', 'A motorcycle with its brake extended standing outside'], 190236: ['An office cubicle with four different types of computers.', 'The home office space seems to be very cluttered.', 'an office with desk computer and chair and laptop.', 'Office setting with a lot of computer screens.', 'A desk and chair in an office cubicle.'], 331352: ['A small closed toilet in a cramped space.', 'A tan toilet and sink combination in a small room.', 'This is an advanced toilet with a sink and control panel.', 'A close-up picture of a toilet with a fountain.', 'Off white toilet with a faucet and controls. '], 517069: ['Two women waiting at a bench next to a street.', 'A woman sitting on a bench and a woman standing waiting for the bus.', 'A woman sitting on a bench in the middle

In [16]:
df=pd.DataFrame(data_dict.items(),columns=['image_id','text_corpus'])

In [17]:
df.iloc[0]['text_corpus']

['A black Honda motorcycle parked in front of a garage.',
 'A Honda motorcycle parked in a grass driveway',
 'A black Honda motorcycle with a dark burgundy seat.',
 'Ma motorcycle parked on the gravel in front of a garage',
 'A motorcycle with its brake extended standing outside']

In [18]:
df.head()

,image_id,text_corpus
0,179765,[A black Honda motorcycle parked in front of a...
1,190236,[An office cubicle with four different types o...
2,331352,"[A small closed toilet in a cramped space., A ..."
3,517069,[Two women waiting at a bench next to a street...
4,182417,[A beautiful dessert waiting to be shared by t...


In [19]:
df.shape

(5000, 2)

In [20]:
def lemmatize_stemm(text):
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocessing(Bigtext):
    
    result = []
    for text in Bigtext:
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                result.append(lemmatize_stemm(token))
    return result

In [21]:
dict_gensim={}
for i in range(df.shape[0]):
    dict_gensim.update({i:preprocessing(df.iloc[i]["text_corpus"])})

In [22]:
print(dict_gensim[0])
#dd=pd.DataFrame(dict_gensim.items(),columns=['id','words'])
dd=pd.Series(dict_gensim)
#dd.drop(columns=['id'],axis=1,inplace=True)

['black', 'honda', 'motorcycle', 'park', 'garage', 'honda', 'motorcycle', 'park', 'grass', 'driveway', 'black', 'honda', 'motorcycle', 'dark', 'burgundy', 'seat', 'motorcycle', 'park', 'gravel', 'garage', 'motorcycle', 'brake', 'extend', 'stand', 'outside']


In [23]:
print(type(dd))
print(dd.shape)
dd.head()

<class 'pandas.core.series.Series'>
(5000,)


0    [black, honda, motorcycle, park, garage, honda...
1    [office, cubicle, different, type, computers, ...
2    [small, close, toilet, cramp, space, toilet, s...
3    [women, wait, bench, street, woman, sit, bench...
4    [beautiful, dessert, wait, share, people, piec...
dtype: object

In [24]:
dictionary = gensim.corpora.Dictionary(dd)     

In [25]:
print(dictionary)

Dictionary(5125 unique tokens: ['black', 'brake', 'burgundy', 'dark', 'driveway']...)


In [26]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 black
1 brake
2 burgundy
3 dark
4 driveway
5 extend
6 garage
7 grass
8 gravel
9 honda
10 motorcycle


In [27]:
#dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=10000)

In [28]:
len(dictionary)

5125

In [29]:
bow_corpus = [dictionary.doc2bow(doc) for doc in dd]

In [30]:
bow_doc_1 = bow_corpus[1]

for i in range(len(bow_doc_1)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0],dictionary[bow_doc_1[i][0]], bow_doc_1[i][1]))

Word 15 ("chair") appears 2 time.
Word 16 ("clutter") appears 1 time.
Word 17 ("computers") appears 1 time.
Word 18 ("cubicle") appears 2 time.
Word 19 ("desk") appears 2 time.
Word 20 ("different") appears 1 time.
Word 21 ("home") appears 1 time.
Word 22 ("laptop") appears 1 time.
Word 23 ("office") appears 5 time.
Word 24 ("screen") appears 1 time.
Word 25 ("set") appears 1 time.
Word 26 ("space") appears 1 time.
Word 27 ("type") appears 1 time.


In [31]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=20, workers=2)

In [52]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.061*"hold" + 0.060*"tennis" + 0.050*"woman" + 0.049*"play" + 0.037*"game" + 0.030*"ball" + 0.025*"young" + 0.024*"stand" + 0.022*"player" + 0.021*"phone"
Topic: 1 
Words: 0.070*"stand" + 0.039*"field" + 0.033*"grass" + 0.026*"tree" + 0.021*"fence" + 0.021*"walk" + 0.018*"giraffe" + 0.017*"near" + 0.016*"lay" + 0.016*"look"
Topic: 2 
Words: 0.062*"water" + 0.060*"beach" + 0.047*"flower" + 0.044*"vase" + 0.039*"kite" + 0.029*"glass" + 0.026*"wine" + 0.023*"fly" + 0.023*"boat" + 0.021*"wave"
Topic: 3 
Words: 0.053*"table" + 0.049*"food" + 0.035*"kitchen" + 0.031*"plate" + 0.023*"bathroom" + 0.021*"sit" + 0.020*"white" + 0.018*"bowl" + 0.017*"toilet" + 0.015*"skateboard"
Topic: 4 
Words: 0.061*"pizza" + 0.039*"street" + 0.034*"park" + 0.028*"horse" + 0.024*"motorcycle" + 0.021*"sit" + 0.021*"plate" + 0.019*"rid" + 0.019*"road" + 0.018*"city"
Topic: 5 
Words: 0.116*"sit" + 0.058*"laptop" + 0.049*"table" + 0.044*"desk" + 0.033*"phone" + 0.021*"woman" + 0.018*"keyboard" + 0

In [53]:
print(lda_model.print_topics(num_topics=10,num_words=7))

[(0, '0.061*"hold" + 0.060*"tennis" + 0.050*"woman" + 0.049*"play" + 0.037*"game" + 0.030*"ball" + 0.025*"young"'), (1, '0.070*"stand" + 0.039*"field" + 0.033*"grass" + 0.026*"tree" + 0.021*"fence" + 0.021*"walk" + 0.018*"giraffe"'), (2, '0.062*"water" + 0.060*"beach" + 0.047*"flower" + 0.044*"vase" + 0.039*"kite" + 0.029*"glass" + 0.026*"wine"'), (3, '0.053*"table" + 0.049*"food" + 0.035*"kitchen" + 0.031*"plate" + 0.023*"bathroom" + 0.021*"sit" + 0.020*"white"'), (4, '0.061*"pizza" + 0.039*"street" + 0.034*"park" + 0.028*"horse" + 0.024*"motorcycle" + 0.021*"sit" + 0.021*"plate"'), (5, '0.116*"sit" + 0.058*"laptop" + 0.049*"table" + 0.044*"desk" + 0.033*"phone" + 0.021*"woman" + 0.018*"keyboard"'), (6, '0.119*"people" + 0.063*"group" + 0.030*"fly" + 0.028*"stand" + 0.025*"sit" + 0.024*"bench" + 0.023*"park"'), (7, '0.055*"sign" + 0.047*"street" + 0.029*"snow" + 0.026*"ski" + 0.022*"person" + 0.018*"stop" + 0.017*"stand"'), (8, '0.071*"clock" + 0.059*"room" + 0.049*"build" + 0.047*"be